In [333]:
from abc import ABC, abstractmethod
import time
import hashlib
import math

from pprint import pprint

### Main functions

In [336]:
def get_current_time() -> float:
    return time.time()


def compute_hash(b: bytes) -> str:
    return hashlib.sha256(string=b, usedforsecurity=True).hexdigest()


def get_merkle_hash(arr: list[str]) -> str:
    result_arr = arr.copy()

    for i in range(len(arr)):
        result_arr[i] = compute_hash(result_arr[i].encode())

    n = int(math.log2(len(arr))) # number of bin graph layers
    for i in range(n):
        new_arr = []
        for i in range(0, len(result_arr), 2):
            s = (result_arr[i] + result_arr[i + 1]).encode()
            h = compute_hash(s)
            new_arr.append(h)
        result_arr = new_arr.copy()
    
    return result_arr[0]

### Account, Transaction and Block

In [350]:
class Account:
    def __init__(self, mnemonic: str, private_key: str):
        self.public_key = compute_hash(f"{mnemonic}{private_key}".encode())

    def __str__(self) -> str:
        return f"<Account '{self.public_key}'>"


class Transaction:
    """PHC - phils coin"""

    def __init__(
        self,
        sender_private_key: str,
        sender_public_key: str,
        reciever_public_key: str,
        PHC: float,
        sender_PHC_total: float,
        reciever_PHC_total: float,
    ):
        self.sender_public_key = sender_public_key
        self.reciever_public_key = reciever_public_key
        self.PHC = PHC
        self.sender_PHC_total = sender_PHC_total
        self.reciever_PHC_total = reciever_PHC_total
        self.timestamp = get_current_time()

        s = sender_public_key + reciever_public_key + str(self.timestamp) + sender_private_key
        signed_hash = compute_hash(s.encode())
        self.hash = signed_hash

    def __str__(self) -> str:
        return f"<Transaction '{self.hash}'>"

    def is_valid(self, sender_private_key: str) -> bool:
        s = self.sender_public_key + self.reciever_public_key + str(self.timestamp) + sender_private_key
        signed_hash = compute_hash(s.encode())
        if self.hash == signed_hash:
            return True
        return False


class Block:
    def __init__(self, index: int, previous_hash: str):
        self.index = index
        self.previous_hash = previous_hash
        self.transactions = []
        self.accounts = []
        self.timestamp = get_current_time()

    def __str__(self) -> str:
        return f"<Block '{self.get_hash()}'>"

    def _set_hash(self):
        self.hash = self.get_hash()

    def get_hash(self) -> str:
        transactions_hash = get_merkle_hash([t.__dict__['hash'] for t in self.transactions])
        s = str(self.index) + self.previous_hash + transactions_hash + str(self.timestamp)
        return compute_hash(s.encode())

### BaseABC and Base

Lets create an abstract class (BaseABC) that will share methods between children.

Lets create Base class that will contain the chain of blocks.

In [351]:
class BaseABC(ABC):
    @property
    @abstractmethod
    def current_block(self) -> Block:
        pass

    @property
    @abstractmethod
    def blocks_count(self) -> int:
        pass

    @abstractmethod
    def _create_block(self, initial: bool) -> Block:
        pass

    @abstractmethod
    def _append_block(self, block: Block):
        pass

    @abstractmethod
    def login(self, mnemonic: str, private_key: str) -> dict[str, str]:
        pass

    @abstractmethod
    def get_account_by_public_key(self, public_key: str):
        pass

    @abstractmethod
    def get_user_PHC_total(self, public_key: str) -> float:
        pass


class Base(BaseABC):
    MAX_TRANSACTIONS_SIZE_IN_BLOCK = 16

    def __init__(self, name: str) -> None:
        self.name = name
        self.chain = []
        self._append_block(self._create_block(initial=True))

    def __str__(self) -> str:
        return f"<Blockchain '{self.name}'>"

### Managers

Lets create blockchain managers

In [352]:
class ChainManager(Base):
    def _append_block(self, block: Block):
        self.chain.append(block)

    def is_valid(self) -> tuple[bool, int]:
        for i in range(self.blocks_count - 1):
            if self.chain[i].hash != self.chain[i].get_hash():
                return False, i

            if i > 0:
                if self.chain[i - 1].hash != self.chain[i].previous_hash:
                    return False, i
        return True, -1


class AccountManager(Base):
    def get_account_by_public_key(self, public_key: str) -> Account | None:
        for block in self.chain:
            for account in block.accounts:
                if public_key == account.public_key:
                    return account
        return

    def get_user_PHC_total(self, public_key: str):
        for block in self.chain[::-1]:
            for transaction in block.transactions[::-1]:
                if transaction.sender_public_key == public_key:
                    return transaction.sender_PHC_total
                elif transaction.reciever_public_key == public_key:
                    return transaction.reciever_PHC_total
        return 0.0

    def login(self, mnemonic: str, private_key: str):
        account = Account(mnemonic=mnemonic, private_key=private_key)
        if self.get_account_by_public_key(public_key=account.public_key):
            return {
                "success": "Login.",
                "public_key": account.public_key,
            }
        return {"error": "Please try again."}

    def add_account(self, mnemonic: str, private_key: str) -> dict[str, str]:
        new_account = Account(mnemonic=mnemonic, private_key=private_key)
        if self.get_account_by_public_key(public_key=new_account.public_key):
            return {
                "error": "Please use another mnemonic. You have a hash collision."
            }
        self.current_block.accounts.append(new_account)
        return {
            "success": "Account has been created.",
            "public_key": new_account.public_key,
        }


class BlockManager(Base):
    @property
    def current_block(self) -> Block:
        return self.chain[-1]

    @property
    def blocks_count(self) -> int:
        return len(self.chain)

    def _create_block(self, initial: bool) -> Block:
        if initial:
            index = 0
            previous_hash = ""
        else:
            index = self.current_block.index + 1
            self.current_block._set_hash()
            previous_hash = self.current_block.hash

        return Block(
            index=index,
            previous_hash=previous_hash,
        )


class TransactionManager(Base):
    def _append_transaction(self, transaction: Transaction):
        self.current_block.transactions.append(transaction)

    def add_transaction(
        self,
        sender_mnemonic: str,
        sender_private_key: str,
        reciever_public_key: str,
        PHC: float,
    ) -> dict[str, str] | Transaction:
        PHC = float(PHC)

        sender_public_key = self.login(
            mnemonic=sender_mnemonic, private_key=sender_private_key
        ).get("public_key")
        if sender_public_key is None:
            return {"error": "Can not login."}

        reciever = self.get_account_by_public_key(public_key=reciever_public_key)
        if not reciever:
            return {"error": "Can not find reciever account."}

        sender_PHC_total = self.get_user_PHC_total(public_key=sender_public_key)
        reciever_PHC_total = self.get_user_PHC_total(public_key=reciever.public_key)

        if sender_PHC_total - PHC < 0:
            return {"error": "You do not have enough PHC."}

        transaction = Transaction(
            sender_private_key=sender_private_key,
            sender_public_key=sender_public_key,
            reciever_public_key=reciever_public_key,
            PHC=PHC,
            sender_PHC_total=sender_PHC_total - PHC,
            reciever_PHC_total=reciever_PHC_total + PHC,
        )

        self._append_transaction(transaction=transaction)
        if (
            len(self.current_block.transactions)
            >= self.MAX_TRANSACTIONS_SIZE_IN_BLOCK
        ):
            block = self._create_block(initial=False)
            self._append_block(block=block)
        return transaction

In [414]:
class Blockchain(AccountManager, BlockManager, TransactionManager, ChainManager):
    def __init__(self, name: str, **kwargs):
        super().__init__(name=name, **kwargs)

### Initialize blockchain

In [415]:
sender_mnemonic = "mnemonic"
sender_private_key = "private_key"
reciever_mnemonic = "mnemonic1"
reciever_private_key = "private_key"

blockchain = Blockchain(name="Phils_blockchain")
print(blockchain)
print()

pprint(vars(blockchain))
print()

pprint(vars(blockchain.chain[0]))

<Blockchain 'Phils_blockchain'>

{'chain': [<__main__.Block object at 0x125f71930>], 'name': 'Phils_blockchain'}

{'accounts': [],
 'index': 0,
 'previous_hash': '',
 'timestamp': 1707073190.989784,
 'transactions': []}


### Create user

In [416]:
sender = blockchain.add_account(mnemonic=sender_mnemonic, private_key=sender_private_key)
print(f"{sender = }\n")

# Get users total coins
sender_public_key = sender.get("public_key")
sender_PHC_total = blockchain.get_user_PHC_total(public_key=sender_public_key)
print(f"{sender_PHC_total = }\n")

# Try to create an account that has the same mnemonic and private_key like user 1
sender_error = blockchain.add_account(
    mnemonic=sender_mnemonic, private_key=sender_private_key
)
print(f"{sender_error = }\n")

# Create second account
reciever = blockchain.add_account(
    mnemonic=reciever_mnemonic, private_key=reciever_private_key
)
print(f"{reciever = }")

sender = {'success': 'Account has been created.', 'public_key': '938b4cb077d33740fd4f7ec42a163c9746aa54ae80a0bf0322cb653cd8f263ec'}

sender_PHC_total = 0.0

sender_error = {'error': 'Please use another mnemonic. You have a hash collision.'}

reciever = {'success': 'Account has been created.', 'public_key': 'e97d2f7b6da84a84692338a2ee49533c32bd861d2f3255b9d5eb7119abed2cee'}


### Create our first transaction

In [417]:
reciever_public_key = reciever.get("public_key")

transaction = blockchain.add_transaction(
    sender_mnemonic=sender_mnemonic,
    sender_private_key=sender_private_key,
    reciever_public_key=reciever_public_key,
    PHC=0,
)
pprint(transaction.__dict__)
print()

# See that our transaction is valid
print(transaction.is_valid(sender_private_key=sender_private_key))

{'PHC': 0.0,
 'hash': '19945ea4c26fcee443405db4d830323c4b45d2689fada6f8fd602a615a6224b7',
 'reciever_PHC_total': 0.0,
 'reciever_public_key': 'e97d2f7b6da84a84692338a2ee49533c32bd861d2f3255b9d5eb7119abed2cee',
 'sender_PHC_total': 0.0,
 'sender_public_key': '938b4cb077d33740fd4f7ec42a163c9746aa54ae80a0bf0322cb653cd8f263ec',
 'timestamp': 1707073196.601423}

True


In [418]:
for _ in range(100):
    blockchain.add_transaction(
        sender_mnemonic=sender_mnemonic,
        sender_private_key=sender_private_key,
        reciever_public_key=reciever_public_key,
        PHC=0,
    )

pprint(blockchain.chain[0].__dict__)
print()

print(blockchain.is_valid())

{'accounts': [<__main__.Account object at 0x125f73700>,
 'hash': '1692aef36a78ed7bbd8ae38d419ec94c080db4f1ee9ad4755307e13ea03a63b9',
 'index': 0,
 'previous_hash': '',
 'timestamp': 1707073190.989784,
 'transactions': [<__main__.Transaction object at 0x125f73640>,
                  <__main__.Transaction object at 0x125f70130>]}

(True, -1)


### Lets make changes in the chain

In [419]:
error_block = blockchain.chain[-2]
blockchain.chain[3] = error_block

print(blockchain.is_valid())

(False, 3)
